In [30]:
import pandas as pd
import re

def roll_num(rollno):
 return re.match(r"\d\d\d\d\w\w\d\d",rollno)
 
 
def ltp_func(ltp_var):
 ltp_dec = ltp_var.split('-')
 return ltp_dec

def list_taken(courses):
 startl = set()
 for i, r in courses.iterrows():
  course = r["subno"]
  if roll_num(r["rollno"]):
   if course not in course_ltp:
    print("feedback not submitted")
   if course_ltp[course][0] != "0":
    startl.add((r["rollno"],r["subno"],"1"))
   if course_ltp[course][1] != "0":
    startl.add((r["rollno"],r["subno"],"2"))
   if course_ltp[course][2] != "0":
    startl.add((r["rollno"],r["subno"],"3"))

 return startl


def list_done (feedback):
 startl = set()
 for i, r in feedback.iterrows():
  if roll_num(r["stud_roll"]):
   startl.add((r["stud_roll"],r["course_code"],str(r["feedback_type"])))
 return startl


feedback = pd.read_csv("course_feedback_submitted_by_students.csv")
course_master = pd.read_csv("course_master_dont_open_in_excel.csv")
student_info = pd.read_csv("studentinfo.csv")
courses = pd.read_csv("course_registered_by_all_students.csv")
output_file = pd.DataFrame(columns = ["rollno","register_sem","schedule_sem","subno","Name","email","aemail","contact"])

sch_sem = {}

for i,r in courses.iterrows():
 if roll_num(r["rollno"]):
  if r["rollno"] not in sch_sem:
   sch_sem[r["rollno"]] = {}

  sch_sem[r["rollno"]][r["subno"]] = (r["register_sem"],r["schedule_sem"])

course_ltp = {}

for i, r in course_master.iterrows():
 course_ltp[r["subno"]] = ltp_func(r["ltp"])

full_list = list_taken(courses)
stud_info = {}

for i,r in student_info.iterrows():
 stud_info[r["Roll No"]] = {}
 stud_info[r["Roll No"]] = r

list_done = list_done(feedback)

full_list = full_list | list_done
rem_list = list_done ^ full_list

for entry in rem_list:
 rollno = entry[0]
 course = entry[1]
 feed_type = entry[2]
 reg_sem = sch_sem[rollno][course][0]
 sched_sem = sch_sem[rollno][course][1]
 if rollno in stud_info:
  name = stud_info[rollno]["Name"]
  mail = stud_info[rollno]["email"]
  amail = stud_info[rollno]["aemail"]
  contact = stud_info[rollno]["contact"]
  new_row = {"rollno":rollno, "register_sem":reg_sem,"schedule_sem":sched_sem,"subno":course,"email":mail,"aemail":amail,"contact":contact,"Name":name};
  output_file = output_file.append(new_row,ignore_index=True)

output_file.to_excel("course_feedback_remaining.xlsx")